# Serialisatrion

In [ ]:
from btclib.ec import S256Point, SecretKey
from btclib.ecds import Signature

- want to store these objects; PublicKey(S256Point), SceretKey and Signatures on disk and possibly transmit over network. 

## Uncompressed SEC format; Standards for efficient cryptography 

In [ ]:
sks = [5000, 2018**4, 0xAFFBEE1234]
for sk in sks:
    print(f"public key for secret key {sk}")
    print(SecretKey(sk=sk).pk.sec(False).hex())
    print(f"Compressed public key for secret key {sk}")
    print(SecretKey(sk=sk).pk.sec(True).hex())

public key for secret key 5000
04ffe558e388852f0120e46af2d1b370f85854a8eb0841811ece0e3e03d282d57c315dc72890a4f10a1481c031b03b351b0dc79901ca18a00cf009dbdb157a1d10
Compressed public key for secret key 5000
02ffe558e388852f0120e46af2d1b370f85854a8eb0841811ece0e3e03d282d57c
public key for secret key 16583822760976
04cd8bdb64c472a6a42e256363215b25888ddf9bb588ccd7cfc33a3611f112fe4a95b3fd785558e9299f94d1a839aed2aac349c6fa1d823641df9897463ca577ee
Compressed public key for secret key 16583822760976
02cd8bdb64c472a6a42e256363215b25888ddf9bb588ccd7cfc33a3611f112fe4a
public key for secret key 755845960244
04021cdbe85013556e1f0555b4a2e96e05f28dbddb0629ca8a8db0f35f51b5bca13b75a3320b21ac91b71be1f114746a6e14e319d9e11d859b26e341f3373a917f
Compressed public key for secret key 755845960244
03021cdbe85013556e1f0555b4a2e96e05f28dbddb0629ca8a8db0f35f51b5bca1


## Transactions

In [ ]:
from btclib.utils import hash256
import requests
from io import BytesIO


class TxFetcher:
    cache = {}

    @classmethod
    def get_url(cls, testnet=False):
        if testnet:
            return "https://testnet.programmingbitcoin.com"
        else:
            return "https://mainnet.programmingbitcoin.com"

    @classmethod
    def fetch(cls, tx_id, testnet=False, fresh=False):
        if fresh or (tx_id not in cls.cache):
            url = f"{cls.get_url(testnet)}/tx/{tx_id}.hex"
            response = requests.get(url)
            try:
                raw = bytes.fromhex(response.text.strip())
            except ValueError:
                raise ValueError(f"Unexpected response: {response.text}")
            if raw[4] == 0:
                raw = raw[:4] + raw[6:]
                tx = Tx.parse(BytesIO(raw), testnet=testnet)
                tx.locktime = int.from_bytes(raw[-4:], "little")
            else:
                tx = Tx.parse(BytesIO(raw), testnet=testnet)
            if tx.id() != tx_id:
                raise ValueError(
                    f"Received unmatching tx ids: {tx.id()} versus {tx_id}"
                )

            cls.cache[tx_id] = tx

        cls.cache[tx_id].testnet = testnet
        return cls.cache[tx_id]


class Tx:

    def __init__(self, version, tx_ins, tx_outs, locktime, testnet=False):
        self.version = version
        self.tx_ins = tx_ins
        self.tx_outs = tx_outs
        self.locktime = locktime
        self.testnet = testnet

    def __repr__(self):
        tx_ins = ""
        for tx_in in self.tx_ins:
            txt_ins += tx_in.__repr__() + "\n"
        tx_outs = ""
        for tx_out in self.tx_outs:
            txt_outs += tx_out.__repr__() + "\n"
        return f"Tx: {self.id()}\nverison: {self.version}\ntx_ins:\n{tx_ins}tx_outs:\n{tx_outs}locktime: {self.locktime}"

    def id(self):
        return self.hash().hex()

    def hash(self):
        return hash256(self.serialize())[::-1]

    def serialize(self):
        result = self.version.to_bytes(4, "little")
        result += encode_varint(len(self.tx_ins))
        for tx_in in self.tx_ins:
            result += tx_in.serialize()

        result += encode_varint(len(self.tx_outs))
        for tx_out in self.tx_outs:
            result += tx_out.serialize()
        result += self.locktime.to_bytes(4, "little")
        return result

    @classmethod
    def parse(cls, s):
        version = s.read(4)
        pass


class TxIn:

    def __init__(self, prev_tx, prev_index, script_sig=None, sequence=0xFFFFFFFF):
        self.prev_tx = prev_tx
        self.prev_index = prev_index
        if script_sig is None:
            self.script_sig = Script()
        else:
            self.script_sig = script_sig
        self.sequence = sequence

    def __repr__(self):
        return f"{self.prev_tx.hex()}:{self.prev_index}"

    def fetch_tx(self, testnet=False):
        return TxFetcher.fetch(self.prev_tx.hex(), testnet=testnet)

    def value(self, testnet=False):
        tx = self.fetch_tx(testnet=testnet)
        return tx.tx_outs[self.prev_index].amount

    def script_pubkey(self, testnet=False):
        tx = self.fetch_tx(self.prev_index)
        return tx.tx_outs[self.prev_index].script_pubkey

    def serialize(self):
        result = self.prev_tx[::-1]
        result += self.prev_index.to_bytes(4, "little")
        result += self.script_sig.serialize()
        result += self.sequence.to_bytes(4, "little")
        return result


class TxOut:

    def __init__(self, amount, script_pubkey):
        self.amount = amount
        self.script_pubkey = script_pubkey

    def __repr__(self):
        return f"{self.amount}:{self.script_pubkey}"

    def serialize(self):

        result = self.amount.to_bytes(8, "little")
        result += self.script_pubkey.serialize()
        return result


def read_varint(s):
    i = s.read(1)[0]
    if i == 0xFD:
        return int.from_bytes(s.read(2), "little")
    elif i == 0xFE:
        return int.from_bytes(s.read(4), "little")
    elif i == 0xFF:
        return int.from_bytes(s.read(8), "little")
    else:
        return i


def encode_varint(i):
    if i < 0xFD:
        return bytes([i])
    elif i < 0x10000:
        return b"\xfd" + i.to_bytes(2, "little")
    elif i < 0x100000000:
        return b"\xfe" + i.to_bytes(4, "little")
    elif i < 0x10000000000000000:
        return b"\xff" + i.to_bytes(8, "little")
    else:
        raise ValueError(f"Integer {i} is too large")

## Script; smart contract programming langauge 

### Operation codes; opcodes

In [6]:
from btclib.utils import hash160, hash256


def op_dup(stack):
    if len(stack) < 1:
        return False
    stack.append(stack[-1])
    return True


def op_hash256(stack):
    if len(stack) < 1:
        return False
    el = stack.pop()  # last in list
    stack.append(hash256(el))
    return True


def op_hash160(stack):
    if len(stack) < 1:
        return False
    el = stack.pop()  # last in list
    stack.append(hash160(el))
    return True


OP_CODE_FUNCTIONS = {
    118: op_dup,
    0xA9: op_hash160,
    170: op_hash256,
}

OP_CODE_NAMES = {}

### Script Parser and serialiser


In [ ]:
import logging

logger = logging.getLogger(__name__)

logger


class Script:

    def __init__(self, cmds=None):
        if cmds is None:
            self.cmds = []
        else:
            self.cmds = cmds

    def __add__(self, other):
        cmds = self.cmds + other.cmds
        return self.__class__(cmds)

    def raw_serialize(self):
        result = b""
        for cmd in self.cmds:
            if type(cmd) == int:
                result += cmd.to_bytes(cmd, 1)
            else:
                length = len(cmd)
                if length < 75:
                    result += length.to_bytes(1, "little")
                elif length > 75 and length < 0x100:
                    result += int(76).to_bytes(1, "little")
                    result += length.to_bytes(1, "little")
                elif length >= 0x100 and length <= 520:
                    result += int(77).to_bytes(1, "little")
                    result += length.to_bytes(2, "little")
                else:
                    #
                    raise ValueError(
                        f"CMD is too long. Max {520} Bytes, received {length} Bytes."
                    )
                result += cmd
        return result

    def serialize(self):
        result = self.raw_serialize()
        total = len(result)
        return encode_varint(total) + result

    def evaluate(self, z):
        cmds = self.cmds[:]
        stack = []
        altstack = []

        while len(cmds) > 1:
            cmd = cmds.pop(0)
            if type(cmd) == int:
                operation = OP_CODE_FUNCTIONS[cmd]
                if cmd in (99, 100):  # OP_IF and OP_NOTIF
                    if not operation(stack, cmds):
                        logger.info(f"Bad op:{OP_CODE_NAMES[cmd]}")
                        return False

                elif cmd in (107, 108):  # OP_TOTALSTACK and OP_FROM ALTSTACK
                    if not operation(stack, altstack):
                        logger.info(f"Bad op:{OP_CODE_NAMES[cmd]}")
                        return False

                elif cmd in (
                    172,
                    173,
                    174,
                    175,
                ):  # OP_CHECKSIG, OP_CHECKSIGVERIFY, OP_CHECKMULTISIG and OP_CHECKMULTISIGVERIFY
                    if not operation(stack, z):
                        logger.info(f"Bad op:{OP_CODE_NAMES[cmd]}")
                        return False
                else:
                    if not operation(stack):
                        logger.info(f"Bad op:{OP_CODE_NAMES[cmd]}")
                        return False
            else:
                stack.append(cmd)
        if len(stack) == 0:
            return False

        if (
            stack.pop() == b""
        ):  # if top element of stack is empty byte (how we store a zero ) we return False
            return False

        return True

    @classmethod
    def parse(cls, s):
        length = read_varint(s)
        cmds = []
        count = 0
        while count < length:
            current = s.read(1)
            count += 1
            current_byte = current[0]
            if current_byte >= 1 and current_byte <= 75:
                n = current_byte
                cmds.append(n)
            elif current_byte == 76:  # OP_PUSHDATA1
                data_length = int.from_bytes(s.read(1), "little")
                cmds.append(s.read(data_length))
                count += data_length + 1
            elif current_byte == 77:  # OP_PUSHDATA2
                data_length = int.from_bytes(s.read(2), "little")
                cmds.append(s.read(data_length))
                count += data_length + 2
            else:
                op_code = current_byte
                cmds.append(op_code)
        if count != length:
            raise SyntaxError("Parsing a script failed")

        return cls(cmds)

169

In [5]:
x = [1, 2, 3, 4, 5]

el = x.pop()

print(x)
print(el)

0x100

[1, 2, 3, 4]
5


256

# 